In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')  # enable import from src/

In [37]:
from pathlib import Path

import gurobipy
from gurobipy import GRB
import dgl
import numpy as np
import torch

from src.data import get_model, get_soc, load_instance

In [59]:
instances = sorted(list(Path('../data/raw').glob('97_9*.jl')))

instance = load_instance(instances[0])

model = get_model(list(range(9)), instance, coupling=True)

model.optimize()

model.ObjVal

1147.0

In [33]:
X = np.array([v.X for v in model.getVars()])
soc_vars_mask = np.array(['soc' in v.getAttr(GRB.Attr.VarName) for v in model.getVars()])
soc = X[soc_vars_mask]
X = X[~soc_vars_mask]

# check that get_soc is working
soc_hat = get_soc(torch.from_numpy(X[None,:]), instance)
torch.isclose(soc_hat[0][1:], torch.from_numpy(soc)).all()

(1746,)